In [1]:
from tensorflow.keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras import layers
import pandas as pd
import numpy as np
import keras
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, KFold
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import Adam
from keras.layers import Dropout
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
data=pd.read_csv('forestfires.csv')
data

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,...,0,0,0,0,1,0,0,0,0,small
1,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,...,0,0,0,0,0,0,0,1,0,small
2,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,...,0,0,0,0,0,0,0,1,0,small
3,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,...,0,0,0,0,1,0,0,0,0,small
4,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,...,0,0,0,0,1,0,0,0,0,small
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,aug,sun,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,...,0,0,0,0,0,0,0,0,0,large
513,aug,sun,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,...,0,0,0,0,0,0,0,0,0,large
514,aug,sun,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,...,0,0,0,0,0,0,0,0,0,large
515,aug,sat,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,...,0,0,0,0,0,0,0,0,0,small


In [3]:
data=data.drop(['month','day'],axis=1)

In [4]:
data

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,dayfri,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.00,1,...,0,0,0,0,1,0,0,0,0,small
1,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.00,0,...,0,0,0,0,0,0,0,1,0,small
2,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.00,0,...,0,0,0,0,0,0,0,1,0,small
3,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.00,1,...,0,0,0,0,1,0,0,0,0,small
4,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.00,0,...,0,0,0,0,1,0,0,0,0,small
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,6.44,0,...,0,0,0,0,0,0,0,0,0,large
513,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,54.29,0,...,0,0,0,0,0,0,0,0,0,large
514,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,11.16,0,...,0,0,0,0,0,0,0,0,0,large
515,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,0.00,0,...,0,0,0,0,0,0,0,0,0,small


In [16]:
mapping = {'small': 1, 'large': 2}

In [17]:
data=data.replace(mapping)

In [18]:
x = np.array(data.iloc[:,0:28])
y = np.array(data.iloc[:,28])

## normalizing data

In [19]:
def norm_func(i):
    x = (i-i.min())/(i.max()-i.min())
    return (x)

In [20]:
x_norm = norm_func(x)

In [21]:
#Split the data
x_train,x_test,y_train,y_test= train_test_split(x_norm,y, test_size=0.2,stratify = y)

## Applying NN

In [22]:
model = Sequential()
model.add(Dense(8, input_dim=28, activation='linear'))
model.add(Dense(4,  activation='tanh'))
model.add(Dense(1,  activation='sigmoid'))

In [23]:
# compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [25]:
history=model.fit(x_train, y_train, validation_split=0.3, epochs=120, batch_size=10)

Epoch 1/120
29/29 [==============================] - 1s 7ms/step - loss: 0.5657 - accuracy: 0.7232 - val_loss: 0.4568 - val_accuracy: 0.7500
Epoch 2/120
29/29 [==============================] - 0s 5ms/step - loss: 0.3467 - accuracy: 0.7232 - val_loss: 0.2564 - val_accuracy: 0.7500
Epoch 3/120
29/29 [==============================] - 0s 4ms/step - loss: 0.1446 - accuracy: 0.7232 - val_loss: 0.0728 - val_accuracy: 0.7500
Epoch 4/120
29/29 [==============================] - 0s 4ms/step - loss: -0.0360 - accuracy: 0.7232 - val_loss: -0.0843 - val_accuracy: 0.7500
Epoch 5/120
29/29 [==============================] - 0s 4ms/step - loss: -0.1919 - accuracy: 0.7232 - val_loss: -0.2175 - val_accuracy: 0.7500
Epoch 6/120
29/29 [==============================] - 0s 4ms/step - loss: -0.3234 - accuracy: 0.7232 - val_loss: -0.3336 - val_accuracy: 0.7500
Epoch 7/120
29/29 [==============================] - 0s 4ms/step - loss: -0.4402 - accuracy: 0.7232 - val_loss: -0.4336 - val_accuracy: 0.7500
Epoch

Epoch 58/120
29/29 [==============================] - 0s 5ms/step - loss: -2.6974 - accuracy: 0.7232 - val_loss: -2.4508 - val_accuracy: 0.7500
Epoch 59/120
29/29 [==============================] - 0s 4ms/step - loss: -2.7326 - accuracy: 0.7232 - val_loss: -2.4835 - val_accuracy: 0.7500
Epoch 60/120
29/29 [==============================] - 0s 4ms/step - loss: -2.7677 - accuracy: 0.7232 - val_loss: -2.5175 - val_accuracy: 0.7500
Epoch 61/120
29/29 [==============================] - 0s 4ms/step - loss: -2.8050 - accuracy: 0.7232 - val_loss: -2.5477 - val_accuracy: 0.7500
Epoch 62/120
29/29 [==============================] - 0s 4ms/step - loss: -2.8394 - accuracy: 0.7232 - val_loss: -2.5813 - val_accuracy: 0.7500
Epoch 63/120
29/29 [==============================] - 0s 5ms/step - loss: -2.8751 - accuracy: 0.7232 - val_loss: -2.6146 - val_accuracy: 0.7500
Epoch 64/120
29/29 [==============================] - 0s 4ms/step - loss: -2.9121 - accuracy: 0.7232 - val_loss: -2.6450 - val_accuracy:

Epoch 115/120
29/29 [==============================] - 0s 5ms/step - loss: -4.7376 - accuracy: 0.7232 - val_loss: -4.2976 - val_accuracy: 0.7500
Epoch 116/120
29/29 [==============================] - 0s 4ms/step - loss: -4.7752 - accuracy: 0.7232 - val_loss: -4.3276 - val_accuracy: 0.7500
Epoch 117/120
29/29 [==============================] - 0s 4ms/step - loss: -4.8104 - accuracy: 0.7232 - val_loss: -4.3594 - val_accuracy: 0.7500
Epoch 118/120
29/29 [==============================] - 0s 4ms/step - loss: -4.8456 - accuracy: 0.7232 - val_loss: -4.3932 - val_accuracy: 0.7500
Epoch 119/120
29/29 [==============================] - 0s 4ms/step - loss: -4.8819 - accuracy: 0.7232 - val_loss: -4.4255 - val_accuracy: 0.7500
Epoch 120/120
29/29 [==============================] - 0s 5ms/step - loss: -4.9169 - accuracy: 0.7232 - val_loss: -4.4592 - val_accuracy: 0.7500


## Accuracy fir train data

In [24]:
scores = model.evaluate(x_train, y_train)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

13/13 [==============================] - 0s 4ms/step - loss: 0.6802 - accuracy: 0.7288
accuracy: 72.88%


## Accuracy for test data

In [26]:
scores = model.evaluate(x_test, y_test)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

4/4 [==============================] - 0s 3ms/step - loss: -4.8026 - accuracy: 0.7308
accuracy: 73.08%
